In [158]:
# program to see if I could detect a threashold for anomalous behavior based one if a decision tree (trained on clean data) 

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation


df_clean = pd.read_csv ('test1.csv')
df_mitm = pd.read_csv ('test_ddos_ping1.csv')

In [159]:
def transform_data(df):
    macs = ['00:0c:29:9d:9e:9e','00:80:f4:09:51:3b','48:5b:39:64:40:79','00:00:29:e6:14:0d']
    df = df[df['MAC Source'].isin(macs)]
    grouped_ip = df.groupby(df['MAC Source'])
    ip_arr_df = [grouped_ip.get_group(d) for d in df['MAC Source'].unique()]
    plt.rcParams["figure.figsize"] = (20,5)
    mac_data = []
    for df in ip_arr_df:
        #print(df['MAC Source'].unique())
        df['Epoch Time'] = pd.to_datetime(df['Epoch Time'], errors='coerce')
        packets_per_min = df.set_index('Epoch Time').groupby(pd.Grouper(freq='1min'))['No.'].count()
        packets_per_min = packets_per_min.fillna(0)

        time_deltas = df.set_index('Epoch Time').groupby(pd.Grouper(freq='1min'))['Delta Time'].mean()
        time_deltas = time_deltas.fillna(time_deltas.mean())
        df_final = pd.merge(packets_per_min, time_deltas, left_index=True, right_index=True)

        df_final['MAC Source'] = df['MAC Source'].unique()[0]

        mac_data.append(df_final)
        #plt.plot(packets_per_min.index, packets_per_min) 
        #print(df)
        #plt.show()
    df = pd.concat(mac_data)
    return df



In [160]:
def return_acc(df):
    X = df[['No.', 'Delta Time']]
    y = df['MAC Source']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test
    clf = DecisionTreeClassifier()
    clf = clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    yeet = metrics.accuracy_score(y_test, y_pred)
    return yeet, clf

def return_acc_test(df, model):
    X = df[['No.', 'Delta Time']]
    y = df['MAC Source']

    y_pred = model.predict(X)
    yeet = metrics.accuracy_score(y, y_pred)
    return yeet

In [161]:
df_clean = transform_data(df_clean)
acc, model = return_acc(df_clean)
acc

1.0

In [162]:
import glob
accuracies = []

files = glob.glob(r'..\test_data\csv2\*.csv')
for file in files:
    df_rest = pd.read_csv(file)
    df_rest = transform_data(df_rest)
    acc = return_acc_test(df_rest, model)

    filename = file.split("\\")[-1].replace('.pcap', '.csv')
    accuracies.append((filename, acc))
    print('finshed', file)

results = pd.DataFrame(accuracies, columns=['Name', 'Accuracy'])
results
    

finshed C:\Users\carlo\Documents\College\reu_cyber\test_data\csv2\eth2dump-clean-0,5h_1.csv
finshed C:\Users\carlo\Documents\College\reu_cyber\test_data\csv2\eth2dump-clean-1h_1.csv
finshed C:\Users\carlo\Documents\College\reu_cyber\test_data\csv2\eth2dump-clean-6h_1.csv
finshed C:\Users\carlo\Documents\College\reu_cyber\test_data\csv2\eth2dump-mitm-change-15m-0,5h_1.csv
finshed C:\Users\carlo\Documents\College\reu_cyber\test_data\csv2\eth2dump-mitm-change-15m-1h_1.csv
finshed C:\Users\carlo\Documents\College\reu_cyber\test_data\csv2\eth2dump-mitm-change-15m-6h_1.csv
finshed C:\Users\carlo\Documents\College\reu_cyber\test_data\csv2\eth2dump-mitm-change-1m-0,5h_1.csv
finshed C:\Users\carlo\Documents\College\reu_cyber\test_data\csv2\eth2dump-mitm-change-1m-1h_1.csv
finshed C:\Users\carlo\Documents\College\reu_cyber\test_data\csv2\eth2dump-mitm-change-1m-6h_1.csv
finshed C:\Users\carlo\Documents\College\reu_cyber\test_data\csv2\eth2dump-mitm-change-30m-1h_1.csv
finshed C:\Users\carlo\Docu

,Name,Accuracy
0,"eth2dump-clean-0,5h_1.csv",1.000000
1,eth2dump-clean-1h_1.csv,1.000000
2,eth2dump-clean-6h_1.csv,0.994444
3,"eth2dump-mitm-change-15m-0,5h_1.csv",0.913978
4,eth2dump-mitm-change-15m-1h_1.csv,0.972678
...,...,...
69,eth2dump-tcpSYNFloodDDoS30m-1h_1.csv,0.666667
70,eth2dump-tcpSYNFloodDDoS30m-6h_1.csv,0.934120
71,"eth2dump-tcpSYNFloodDDoS5m-0,5h_1.csv",0.860215
72,eth2dump-tcpSYNFloodDDoS5m-1h_1.csv,0.928962


In [165]:
results.to_csv('results3.csv')